In [ ]:
# importing all the libraries
!pip install catboost
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import re
from sklearn.model_selection import cross_val_score
from catboost import CatBoostRegressor, Pool, cv

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/cassandra/
root = "/content/drive/MyDrive/cassandra/"
root = ""

[Errno 2] No such file or directory: '/content/drive/MyDrive/cassandra/'
/content


In [ ]:
# load the training and testing data frames
train_df = pd.read_csv(root + 'trainData.csv')
test_df = pd.read_csv(root + 'testData.csv')

FileNotFoundError: ignored

In [ ]:
#train_df.rename(columns={"Number_of_Days_until_Payment":"target"}, inplace=True)
#test_df.rename(columns={"Number_of_Days_until_Payment":"target"}, inplace=True)

In [ ]:
train_df.dtypes
# taking a look at features and their types

In [ ]:
print(train_df.isna().sum(), '\n')
print(test_df.isna().sum())
# check if data has any NaN values

In [ ]:
train_df.Description.fillna('', inplace=True)
test_df.Description.fillna('', inplace=True)
# samples with no Description have NaN values so we replace them with empty strings.

In [ ]:
plt.figure(figsize=(16,10))
sns.kdeplot(train_df.Number_of_Days_until_Payment)
plt.savefig('fig4')
# taking a look at output distribution, we notice that most target values lie within a reasonable range
# and outliers gradually decrease.

In [ ]:
train_df = train_df.drop(index = train_df[train_df.Number_of_Days_until_Payment < 0].index).reset_index(drop=True)
# dropping the samples with negative target values

In [ ]:
# converting string dates into datetime format
train_df['Invoice_Date'] = pd.to_datetime(train_df['Invoice_Date'])
train_df['Due_Date'] = pd.to_datetime(train_df['Due_Date'])
train_df['Created'] = pd.to_datetime(train_df['Created'])

test_df['Invoice_Date'] = pd.to_datetime(test_df['Invoice_Date'])
test_df['Due_Date'] = pd.to_datetime(test_df['Due_Date'])
test_df['Created'] = pd.to_datetime(test_df['Created'])

In [ ]:
train_df['Due_Invoice_delta'] = (train_df['Due_Date'] - train_df['Invoice_Date']).astype('timedelta64[D]').astype(int)
test_df['Due_Invoice_delta'] = (test_df['Due_Date'] - test_df['Invoice_Date']).astype('timedelta64[D]').astype(int)
# take difference of due date and invoice date to make a new feature

In [ ]:
date_cols = ['Invoice_Date', 'Created', 'Due_Date']
for col in date_cols:
  train_df[col + "_day"] = train_df[col].dt.day.astype(str)
  train_df[col + "_month"] = train_df[col].dt.month.astype(str)
  train_df[col + "_year"] = train_df[col].dt.year.astype(str)
  train_df[col + "_weekday"] = train_df[col].dt.weekday.astype(str)

  test_df[col + "_day"] = test_df[col].dt.day.astype(str)
  test_df[col + "_month"] = test_df[col].dt.month.astype(str)
  test_df[col + "_year"] = test_df[col].dt.year.astype(str)
  test_df[col + "_weekday"] = test_df[col].dt.weekday.astype(str)

train_df.drop(columns=date_cols, inplace=True)
test_df.drop(columns=date_cols, inplace=True)

# split date columns into day, month, year and weekday

In [ ]:
train_description= train_df['Description'].to_list()
test_description = test_df['Description'].to_list()

def text_preprocessing(
    texts:list,
    punctuations = r'''/!()-[]{};:'"\,<>./?@#$%^&*_“~''',
    stop_words=["and", "is", 'a', 'the', 'in', 'be', 'will','am', 'are', 'to', 'on', 'because', 'since', 'of', 'an', 'as', 'at', 'by', 'from', 'has', 'he', 'it', 'its', 'that', 'was', 'were', 'with']
    )->list:
  
  final_text=[]
  # print(text)
  for text in texts:
    for x in text.lower(): 
        if x in punctuations: 
            text = text.replace(x, "  ")
      # Setting every word to lower
    text = text.lower()

    # Converting all our text to a list 
    text = text.split(' ')

    # Droping empty strings
    text = [x for x in text if x!='']

    # Droping stop words
    text = [x for x in text if x not in stop_words]
    final_text.append(text)

  return final_text

In [ ]:
final_description= text_preprocessing(train_description)
final_test_description= text_preprocessing(test_description)

train_df['Description'] = [" ".join(s) for s in final_description]
test_df['Description'] = [" ".join(s) for s in final_test_description]

document = list(train_df['Description'])
document2 = list(test_df['Description'])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Create a Vectorizer Object
vectorizer = CountVectorizer()

# Encode the Document

new_df = list(pd.concat([train_df.drop(columns='Number_of_Days_until_Payment'), test_df], axis=0)['Description'])

vectorizer.fit(new_df)
vector = vectorizer.transform(document)
vector = vectorizer.transform(document2)

ar = np.array(vector.toarray())
at = np.array(vector.toarray())

In [ ]:
# finally add numerically encoded text features in the data frames
a = pd.DataFrame(ar)
at = pd.DataFrame(at)
train_df = pd.concat([train_df, a], axis=1)
test_df = pd.concat([test_df, at], axis=1)

dic = {}
c=0
for j, s in enumerate(set(train_df['Description'])):
  dic[''.join(sorted(s))] = j
  c=j
for j, s in enumerate(set(test_df['Description'])):
  dic[''.join(sorted(s))] = j+c


In [ ]:
train_df['D_size'] = train_df['Description'].apply(lambda s: len(s.split()))
test_df['D_size'] = test_df['Description'].apply(lambda s: len(s.split()))
# we make a new feature by counting the number of words in the Description of the sample

In [ ]:
#train_df.drop(columns='Description', inplace=True)
#test_df.drop(columns='Description', inplace=True)

In [ ]:
train_df['Outstanding_zero'] = train_df['Outstanding'].apply(lambda x: (1 if x==0 else 0))
test_df['Outstanding_zero'] = test_df['Outstanding'].apply(lambda x: (1 if x==0 else 0))

# we made a new feature called “Outstandind_zero” which was 1 if the column was zero and 0 otherwise

In [ ]:
train_df['Outstanding_Amount_ratio'] = train_df['Outstanding']/train_df['Amount']
train_df['Settled_Amount_ratio'] = train_df['Settled']/train_df['Amount']
train_df['Outstanding_Settled_ratio'] = train_df['Outstanding']/train_df['Settled']

test_df['Outstanding_Amount_ratio'] = test_df['Outstanding']/test_df['Amount']
test_df['Settled_Amount_ratio'] = test_df['Settled']/test_df['Amount']
test_df['Outstanding_Settled_ratio'] = test_df['Outstanding']/test_df['Settled']

# We took ratios of these three columns to create three new features.

In [ ]:
num_cols = ['Amount', 'Settled', 'Outstanding', 'Due_Invoice_delta', 'Outstanding_zero', 'Outstanding_Amount_ratio', 'Settled_Amount_ratio', 'Outstanding_Settled_ratio']

In [ ]:
train_df

In [ ]:
new_df = pd.concat([train_df.drop(columns='Number_of_Days_until_Payment'), test_df], axis=0)
for col in num_cols:
  train_df[col + '_mean'] = train_df['Vendor_Name'].map(dict(new_df.groupby('Vendor_Name')[col].mean()))
  train_df[col + '_median'] = train_df['Vendor_Name'].map(dict(new_df.groupby('Vendor_Name')[col].median()))
  train_df[col + '_std'] = train_df['Vendor_Name'].map(dict(new_df.groupby('Vendor_Name')[col].std(ddof=0)))
  train_df[col + '_count'] = train_df['Vendor_Name'].map(dict(new_df.groupby('Vendor_Name')[col].count()))
  train_df[col + '_min'] = train_df['Vendor_Name'].map(dict(new_df.groupby('Vendor_Name')[col].min()))
  train_df[col + '_max'] = train_df['Vendor_Name'].map(dict(new_df.groupby('Vendor_Name')[col].max()))
  
  test_df[col + '_mean'] = test_df['Vendor_Name'].map(dict(new_df.groupby('Vendor_Name')[col].mean()))
  test_df[col + '_median'] = test_df['Vendor_Name'].map(dict(new_df.groupby('Vendor_Name')[col].median()))
  test_df[col + '_std'] = test_df['Vendor_Name'].map(dict(new_df.groupby('Vendor_Name')[col].std(ddof=0)))
  test_df[col + '_count'] = test_df['Vendor_Name'].map(dict(new_df.groupby('Vendor_Name')[col].count()))
  test_df[col + '_min'] = test_df['Vendor_Name'].map(dict(new_df.groupby('Vendor_Name')[col].min()))
  test_df[col + '_max'] = test_df['Vendor_Name'].map(dict(new_df.groupby('Vendor_Name')[col].max()))

# we took mean, median, minimum, maximum, stddev and count of target for each unique Vendor and made these into new features.

In [ ]:
plt.figure(figsize=(16,10))
plt.rcParams.update({'font.size': 22})
sns.kdeplot(train_df[train_df['Amount'] < 1e3]['Amount'], train_df['Number_of_Days_until_Payment'])
plt.savefig('fig1')

In [ ]:
x = ['Monday', 'Tuesday', 'Wednusday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
y= train_df.groupby('Invoice_Date_weekday')['Number_of_Days_until_Payment'].median().values
plt.figure(figsize=(16,10))
sns.barplot(x,y)
plt.ylabel('Number_of_Days_until_Payment')
plt.xlabel('Invoice Weekday')
plt.savefig('fig2')

In [ ]:
x = ['Jan', 'Feb', 'Mar', 'Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
y = train_df.groupby('Invoice_Date_month')['Number_of_Days_until_Payment'].median().values
plt.figure(figsize=(16,10))
sns.barplot(x,y)

In [ ]:
plt.figure(figsize=(16,10))
sns.scatterplot(train_df['Due_Invoice_delta'], train_df['Number_of_Days_until_Payment'], color='red')
plt.savefig('fig3')

In [ ]:
train_df['']

In [ ]:
x_train = np.array(train_df.drop(columns='Number_of_Days_until_Payment'))
y_train = np.array(train_df['Number_of_Days_until_Payment'])
x_test = np.array(test_df)

In [ ]:
# training the CatBoost Regressor
date_cols = ['Invoice_Date', 'Created', 'Due_Date']
a = []
for col in date_cols:
    a.append(col + "_day")
    a.append(col + "_month")

    a.append(col + "_year")

    a.append(col + "_weekday")

a.append("Vendor_Name")

pool_train = Pool(train_df.drop(columns=['Number_of_Days_until_Payment']), train_df['Number_of_Days_until_Payment'],
                  cat_features = a)
model = CatBoostRegressor(verbose=0)
model.fit(pool_train)

In [ ]:
print(cv(pool_train, model.get_all_params(),fold_count=5, plot=True))
# k fold cross validation to evaluate our model

In [ ]:
np.sqrt(np.mean((y_train - model.predict(x_train))**2))
# checking the RMSE score

In [ ]:
# generating final predictions
pred = model.predict(test_df)
out = pd.DataFrame([pd.read_csv(root + 'testData.csv')['Vendor_Name'], pred]).transpose()
out.columns = ['Vendor_Name', 'Number_of_Days_until_Payment']
out.to_csv('out1.csv', index=False)
pd.read_csv('out1.csv')